In [7]:
import os
import sys
from dotenv import load_dotenv
import pandas as pd

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


load_dotenv()

True

In [3]:
from src.utils import load_dataset
df = load_dataset("df_with_llmkeyphrases")
df = df[["abstract", "yake_keywords", "keyphrases"]]

# examples = [0, 25, 37, 61, 107]

Данные успешно загружены из ../data/df_with_llmkeyphrases.parquet. Количество записей: 200


In [ ]:
# display(df.loc[examples])
# df.loc[examples].to_csv("../data/selected_examples.csv", index=False)

,abstract,yake_keywords,keyphrases
0,Представлен усовершенствованный метод парных с...,"[котором посредством табличных, посредством та...","[усовершенствованный метод парных сравнений, л..."
25,Статья посвящена разработке комплексной модели...,"[Статья посвящена разработке, посвящена разраб...","[комплексная модель диктора, текстонезависимая..."
37,В работе рассмотрена обобщенная модель образов...,"[образования новой фазы, переходе первого рода...","[образование новой фазы, фазовый переход перво..."
61,Предложено обобщение класса экспоненциально-ст...,"[обобщенных распределений Лапласа, Предложено ...","[экспоненциально-степенные распределения, обоб..."
107,Рассматривается задача объединения графов с об...,"[Petri Nets Tools, Colored Petri Nets, пакета ...","[Объединение графов, Сети Петри, Общая часть г..."


In [9]:
df = pd.read_csv("../data/selected_examples.csv")
df.head()

,abstract,yake_keywords,keyphrases
0,Представлен усовершенствованный метод парных с...,['котором посредством табличных' 'посредством ...,['усовершенствованный метод парных сравнений' ...
1,Статья посвящена разработке комплексной модели...,['Статья посвящена разработке' 'посвящена разр...,['комплексная модель диктора' 'текстонезависим...
2,В работе рассмотрена обобщенная модель образов...,['образования новой фазы' 'переходе первого ро...,['образование новой фазы' 'фазовый переход пер...
3,Предложено обобщение класса экспоненциально-ст...,['обобщенных распределений Лапласа' 'Предложен...,['экспоненциально-степенные распределения'\n '...
4,Рассматривается задача объединения графов с об...,['Petri Nets Tools' 'Colored Petri Nets' 'паке...,['Объединение графов' 'Сети Петри' 'Общая част...


Я отобрал примеры, я вывел их в отдельный selected_samples файл, так как выборка из 200 абстрактов выполняется случайно, я не стал ставить конкретные сиды на генерацию и на отбор. Тем не менее выбранные показатели довольно легко находятся при беглом просмотре с любой конфигурацией, ведь паттерн одинаков, как и соотвествующие выовды:

1) +LLM Семантика. LLM выдает чистые и осмысленные концепции, в то время как YAKE давал много артефактов, включая академический шум в виде "в статье рассматрвиается"
2) +LLM LLM хорошо обобщает и находит релевантные ключевые фразы. Статистический метод на это не способен
3) +Yake YAKE! все таки смог найти некоторые четкие уникальные термины, это видно в последнем пятом примере (Petri Nets Tools), однако эти уникальные термины могут быть лишь просто уникальным упоминанием, а не действительно ключевым словом.
4) +YAKE Статистический метод справляется с 100 и более аннотациями в секунду и стоит практически ничего в денежном эквиваленте, в то время как использование LLM при моих параметрах батчинга 20 и 10 одновременных запросах способен только на примерно 10 аннотаций в секунду
5) +YAKE реализация этого метода довольно проста, в то время как для LLM нужно построить небольшой пайплайн. 

| Критерий | YAKE (Классический подход) | LLM (Gemini 2.5 Flash Lite) |
| :--- | :--- | :--- |
| **Качество фраз** | Низкое-Среднее (извлекает n-граммы, шум, "рваные" фразы) | Высокое (извлекает семантические концепции, обобщает) |
| **Скорость** | Очень высокая (<1с на ~200 док.) | Низкая (20с на 200 док.) |
| **Стоимость** | 0 (бесплатно) | Низкая, но не нулевая (~$0.01 за 200 док.) |
| **Сложность реализации** | Низкая (импорт библиотеки) | Средняя (требует инженерии API, обработки ошибок) |
| **Сценарий использования**| Быстрый baseline, поиск по точным терминам, прототипирование. | Production-системы, семантический анализ, задачи, где качество важнее скорости. |

Выбор метода зависит от задачи: для быстрого прототипа и простого извлечения терминов YAKE является адекватным baseline, но для построения качественной системы, понимающей смысл текста, LLM-подход является наилучшим